In [1]:
import time as t
import datetime as dt
import pandas as pd
import os
import sys
sys.path.insert(1, '../')
from utils.common import *

In [82]:
import pickle
import ta

In [ ]:
### test full flow
test = pd.read_csv('new_all_calls.csv')
messages = test.loc[test['text'].fillna('11').str.contains('AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy')].copy()
messages['date'] = pd.to_datetime(messages['date'])

In [ ]:
pattern = r'`([1-9A-HJ-NP-Za-km-z]{32,44})`'
messages['address'] = messages['text'].str.extract(pattern)
messages['expected_launch_time'] = messages['text'].str.extract(r'(?<=Launch:\*\* )(.*)')[0].str.strip('`')
messages['mcap'] = messages['text'].str.extract(r'(?<=Mcap:\*\* )(.*)')[0].str.strip('`')
messages['liq'] = messages['text'].str.extract(r'(?<=Liq:\*\* )(.*)')[0].str.strip('`')
messages['s_mm'] = messages['text'].str.extract(r'(?<=Mutable Metadata: )(.*)')
messages['s_ma'] = messages['text'].str.extract(r'(?<=Mint Authority: )(.*)')
messages['s_fa'] = messages['text'].str.extract(r'(?<=Freeze Authority: )(.*)')
messages['s_s'] = messages['text'].str.extract(r'(?<=Score: )(.*)')
messages.loc[messages['s_mm'].str.contains('Yes'), 's_mm2'] = True
messages.loc[messages['s_mm'].str.contains('No'), 's_mm2'] = False
messages.loc[messages['s_ma'].str.contains('Yes'), 's_ma2'] = True
messages.loc[messages['s_ma'].str.contains('No'), 's_ma2'] = False
messages.loc[messages['s_fa'].str.contains('Yes'), 's_fa2'] = True
messages.loc[messages['s_fa'].str.contains('No'), 's_fa2'] = False
messages.loc[messages['s_s'].str.contains('Bad'), 's_q'] = 0
messages.loc[messages['s_s'].str.contains('Neutral'), 's_q'] = 1
messages.loc[messages['s_s'].str.contains('Good'), 's_q'] = 2
messages['s_sni'] = messages['s_s'].str.extract(r'([0-9]+)')
messages['s_sni'] = messages['s_sni'].fillna(0)

mult = {'K': 1e3, 'M': 1e6, 'B': 1e9, 'T': 1e12}
messages['mcap'] = messages['mcap'].str.strip('[$\*]')
messages['mcap_num'] = messages['mcap'].str.extract('([0-9\.]+)').astype(float) * messages['mcap'].str.extract(
    '([A-Z])').replace(mult).fillna(1)
messages['liq'] = messages['liq'].str.strip(r" \[.*\]")
messages['liq_num'] = messages['liq'].str.extract('([0-9\.]+)').astype(float) * messages['liq'].str.extract(
    '([A-Z])').replace(mult).fillna(1)
messages[['s_q', 'mcap_num', 'liq_num']] = messages[['s_q', 'mcap_num', 'liq_num']].astype(int)
tp = messages['expected_launch_time'].str.extractall('([0-9]+)').unstack()
tp.columns = tp.columns.droplevel(0)
tp['hours'] = messages['expected_launch_time'].str.contains('hour')
tp['sec'] = messages['expected_launch_time'].str.contains('second')
tp.loc[tp['hours'] == True, 'new'] = tp.loc[tp['hours'] == True].apply(
    lambda x: dt.timedelta(hours=int(x[0]), minutes=int(x[1])), axis=1)
tp.loc[tp['sec'] == True, 'new'] = tp.loc[tp['sec'] == True].apply(lambda x: dt.timedelta(seconds=int(x[0])),
                                                                    axis=1)
tp.loc[tp['new'].isna(), 'new'] = tp.loc[tp['new'].isna()].apply(lambda x: dt.timedelta(minutes=int(x[0])),
                                                                    axis=1)
tp.loc[tp['new'] >= dt.timedelta(days=30), 'new'] = dt.timedelta(seconds=0)

messages = pd.concat([messages, tp[['new']]], axis=1)
messages.loc[(messages['expected_launch_time'].str.contains('ago')), 'expected_launch_time_ts'] = messages['date'] - pd.to_timedelta(messages['new'])
messages.loc[(messages['expected_launch_time'].str.contains('In')), 'expected_launch_time_ts'] = messages['date'] + pd.to_timedelta(messages['new'])
messages = messages.drop(columns=['s_mm', 's_ma', 's_fa', 's_s', 'mcap', 'liq', 'text', 'new'])

In [ ]:
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT DISTINCT address FROM calls" 
    tracked_a = pd.read_sql_query(query, conn)
    message_clean = messages.loc[~messages['address'].isin(tracked_a['address'].unique())]
    message_clean.to_sql('calls', conn, if_exists='append', index=False)

In [29]:
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT * FROM calls"  # WHERE launched is NULL
    df = pd.read_sql_query(query, conn)

In [30]:
df

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,buy,buy_time,buy_price
0,212902,2024-04-16 09:44:54,AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,27 seconds ago,2024-04-16 09:44:27,0,0,0,0,2,1270,1780,NaN,None,None,None,None
1,215618,2024-04-18 14:35:57,CfQSSRk6Md2cnZCzkJu2QjAVua5FzGMCua7CrhQGRURK,In 1 minute,2024-04-18 14:36:57,0,0,0,2,0,9050,18110,1.0,2024-04-18 14:37:00,None,None,None
2,215619,2024-04-18 14:37:11,GKMRt3xZznGK4wxoBw3SZfMTwVR9LLFTJxbqMCUvMabg,11 seconds ago,2024-04-18 14:37:00,1,0,0,0,2,1390,2500,1.0,2024-04-18 14:37:00,None,None,None
3,215620,2024-04-18 14:37:58,66Yp2kKFLsEjszAMVttwHFDZRZRyGt9kbKoXnuv99Dx3,36 seconds ago,2024-04-18 14:37:22,1,0,0,0,4,1730,1390,1.0,2024-04-18 14:42:00,None,None,None
4,215621,2024-04-18 14:38:20,Ecz4mak5jScXrwZkMGJB3mqqpeuJPWPTddJW6Rgmt9Za,27 seconds ago,2024-04-18 14:37:53,0,0,0,0,2,1110,1110,1.0,2024-04-18 14:38:00,None,None,None
5,215622,2024-04-18 14:39:15,BdSSmAmWAE8F2yhjAPW91Boa4fQfu98EKt3xxgLKQVxn,36 seconds ago,2024-04-18 14:38:39,1,1,0,0,2,40230,80460,1.0,2024-04-18 14:39:00,None,None,None
6,218927,2024-04-21 01:14:29,HbzH8fFYQRLjHgDK7G7qXYzWdV4fnwTSdb71zAZvJyDS,14 seconds ago,2024-04-21 01:14:15,0,0,0,2,0,1650,3290,1.0,2024-04-21 01:14:00+00:00,None,None,None
7,218928,2024-04-21 01:14:51,6xPz9dztCA6TdMsPZ6AHAZ6NEdGMjrouLb3fov8vbBJF,476017 hours and 14 minutes ago,2024-04-21 01:14:51,0,0,0,0,1,770,1500,1.0,2024-04-21 01:14:00,None,None,None
8,218929,2024-04-21 01:16:28,6XejiuxHwr31q7iZr4bYWzuNzzKoQWh4X4rrYP1qkt6t,13 seconds ago,2024-04-21 01:16:15,1,1,0,0,3,1490,2990,1.0,2024-04-21 01:16:00,None,None,None
9,218930,2024-04-21 01:17:25,7VEqMtsR2bRFU3g5cMTHuCYHdR6x1cwoVfoQ1uenJVqs,476017 hours and 17 minutes ago,2024-04-21 01:17:25,0,0,0,2,0,57080,23620,1.0,2024-04-21 01:17:00,None,None,None


In [179]:
def get_static_data(token):
    with SQLiteDB('../dbs/calls.db') as conn:
        query = f'SELECT * FROM calls WHERE address="{token}"'
        df = pd.read_sql_query(query, conn)
        static_features = ['s_mm2','s_ma2','s_fa2','s_q','s_sni','mcap_num','liq_num']
        static_data = df.loc[:,['address']+static_features]
        return static_data

In [201]:
static_data = get_static_data('AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy')
static_data

,address,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num
0,AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,0,0,0,0,2,1270,1780


In [230]:
def get_ochl_data(token, launch_time):
    while True:
        ochl_data = get_price_data(token, int(launch_time.timestamp()), int((launch_time + dt.timedelta(minutes=20)).timestamp()))
        if ochl_data.shape[0] == 0:
            print('Incorrect Launch time')
            return None
        elif ochl_data.shape[0] < 21:
            diff = 21 - ochl_data.shape[0]
            print('Too early. Waiting', diff, 'minutes')
            t.sleep(60*diff)
        return ochl_data

In [232]:
lt = dt.datetime.fromtimestamp(1713260700)
ochl_data = get_ochl_data('AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy', lt)

In [233]:
ochl_data.head(5)

,address,c,h,l,o,type,unixTime,v
0,AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,0.000020,0.000069,0.000017,0.000020,1m,1713260700,7.919336e+08
1,AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,0.000028,0.000055,0.000019,0.000020,1m,1713260760,5.525451e+08
2,AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,0.000034,0.000040,0.000024,0.000028,1m,1713260820,4.334747e+08
3,AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,0.000041,0.000056,0.000030,0.000034,1m,1713260880,3.165894e+08
4,AZwF5h6uKHxp8ta27ba4t3S5LQ9JdwHmkeae22ACDgqy,0.000047,0.000056,0.000032,0.000041,1m,1713260940,2.704976e+08


In [185]:
def prepare_for_ml(static_data, ochl_data):
    ## temp fix to make ta work...
    prepared_data = pd.concat([ochl_data,pd.DataFrame({'address':'dummy'}, index=[21])])

    ## adjustments
    prepared_data['v_usd'] = prepared_data['v']*prepared_data['c']
    prepared_data['time'] = (prepared_data['unixTime'] - prepared_data.groupby('address')['unixTime'].transform(min))/60
    prepared_data = pd.merge(prepared_data, 
                        prepared_data.drop_duplicates(subset='address',keep='first')[['address','o','c','h','l']],
                        on='address', how='left', suffixes=['','_first'])
    for i in ['o','c','h','l']:
        prepared_data['normal_'+i] = prepared_data[i]/prepared_data[i+'_first']

    ##simple checks
    def accumul(x):
        last_v0c = 0
        res = []
        for v in x:
            if v == 0:
                cur_v0c = last_v0c + 1
            elif v != 0:
                cur_v0c = 0
            res += [cur_v0c]
            last_v0c = cur_v0c
        return res
    prepared_data['v_usd_0conseq'] = prepared_data.groupby('address')['v_usd'].transform(accumul)
    prepared_data['target_check'] = np.where((prepared_data['v_usd_0conseq']>=1) & (prepared_data['time']<=20), 0, 1)
    prepared_data['normal_c_delta'] = prepared_data['normal_c'] / prepared_data.groupby('address')['normal_c'].shift(1)
    prepared_data['range'] = prepared_data['normal_h']/prepared_data['normal_l']
    prepared_data['ath_c'] = prepared_data.groupby('address')['normal_c'].cummax()
    prepared_data['atl_c'] = prepared_data.groupby('address')['normal_c'].cummin()
    prepared_data['ath_h'] = prepared_data.groupby('address')['normal_h'].cummax()
    prepared_data['atl_l'] = prepared_data.groupby('address')['normal_l'].cummin()
    prepared_data['ath_c_share'] = prepared_data['normal_c']/prepared_data['ath_c']
    prepared_data['atl_c_share'] = prepared_data['normal_c']/prepared_data['atl_c']
    prepared_data['ath_h_share'] = prepared_data['normal_c']/prepared_data['ath_h']
    prepared_data['atl_l_share'] = prepared_data['normal_c']/prepared_data['atl_l']

    ##indicators
    prepared_data['ema'] = prepared_data.groupby('address').apply(lambda x: ta.trend.ema_indicator(close=x['normal_c'], window=3, fillna=True)).reset_index(level=0, drop=True)
    prepared_data['bbh'] = prepared_data.groupby('address').apply(lambda x: ta.volatility.bollinger_hband_indicator(x["c"], window=3, window_dev=2, fillna=True)).reset_index(level=0, drop=True)
    prepared_data['bbl'] = prepared_data.groupby('address').apply(lambda x: ta.volatility.bollinger_lband_indicator(x["c"], window=3, window_dev=2, fillna=True)).reset_index(level=0, drop=True)
    prepared_data['mfi'] = prepared_data.groupby('address').apply(lambda x: ta.volume.money_flow_index(high = x["h"], low= x['l'], close=x['c'], volume=x['v'], window=3, fillna=True)).reset_index(level=0, drop=True)
    prepared_data['accdist'] = prepared_data.groupby('address').apply(lambda x: ta.volume.acc_dist_index(high = x["h"], low= x['l'], close=x['c'], volume=x['v'], fillna=True)).reset_index(level=0, drop=True)
    prepared_data['vwap'] = prepared_data.groupby('address').apply(lambda x: ta.volume.volume_weighted_average_price(high = x["h"], low= x['l'], close=x['c'], volume=x['v'], fillna=True)).reset_index(level=0, drop=True)
    prepared_data['rsi'] = prepared_data.groupby('address').apply(lambda x: ta.momentum.rsi(close=x['c'], window=3, fillna=True)).reset_index(level=0, drop=True)
    prepared_data['stoch'] = prepared_data.groupby('address').apply(lambda x: ta.momentum.stoch(high = x["h"], low= x['l'], close=x['c'], window=3, smooth_window=3 , fillna=True)).reset_index(level=0, drop=True)
    prepared_data['stochs'] = prepared_data.groupby('address').apply(lambda x: ta.momentum.stoch_signal(high = x["h"], low= x['l'], close=x['c'], window=3, smooth_window=3 , fillna=True)).reset_index(level=0, drop=True)
    prepared_data['stochrsi'] = prepared_data.groupby('address').apply(lambda x: ta.momentum.stochrsi(close=x['c'], window=3, smooth1=3, smooth2=3 , fillna=True)).reset_index(level=0, drop=True)

    ## cleanup
    final_df = prepared_data.loc[(prepared_data['time']<=20)]
    if final_df['target_check'].min() == 0:
        print('drop') ## return NONE!!
    final_df = final_df.drop(columns=['type','h','o','l','c','unixTime','v','o_first', 'c_first', 'h_first', 'l_first','target_check'])

    ## formating
    final_df['time'] = final_df['time'].astype(int)
    final_df['normal_c_delta'] = final_df['normal_c_delta'].fillna(1)
    final_df = pd.pivot_table(final_df, index=['address'], columns=['time']).reset_index()
    final_df.columns = [col[0]+'_'+str(col[1]) for col in final_df.columns.values]
    final_df = pd.merge(final_df.rename(columns={'address_':'address'}),
                        static_data,
                        on='address',
                        how='left')
    
    return final_df

In [204]:
final_df = prepare_for_ml(static_data, ochl_data)

In [209]:
def make_predictions(final_df):
    model_clf = pickle.load(open('../models/model_clf.sav', 'rb'))
    model_regr = pickle.load(open('../models/model_regr.sav', 'rb'))
    SCAM_PREDICTION = model_clf.predict(final_df.drop(columns='address').rename(columns={'mcap_num':'Mcap_num','liq_num':'Liq_num'}))[0]
    X_PREDICTION = model_regr.predict(final_df.drop(columns='address').rename(columns={'mcap_num':'Mcap_num','liq_num':'Liq_num'}))[0]
    return SCAM_PREDICTION, X_PREDICTION

In [210]:
make_predictions(final_df)

/Users/gudkovm/Projects/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [21:43:12] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/gbtree.cc:385: Changing updater from `grow_gpu_hist` to `grow_quantile_histmaker`.
  warnings.warn(smsg, UserWarning)


(0, 6.131664)